In [22]:
from sympy.physics.vector import (
    ReferenceFrame,
    dynamicsymbols,
    get_motion_params
)
from sympy import symbols, Piecewise, Eq, solveset, Min, Max, latex

from IPython.display import Markdown
import numpy as np
import plotly.graph_objects as go

In [2]:
T = symbols('T', nonnegative=True)
R = symbols('R', nonnegative=True)
t = symbols('t', nonnegative=True)

In [3]:
N = ReferenceFrame('N')

In [4]:
a_magnitude = symbols('|a|', nonnegative=True)
v0_magnitude = symbols('|v_{0}|', nonnegative=True) # TODO: v0 should be a 3D vector, and accept negative values
p0_magnitude = symbols('|p_{0}|', nonnegative=True) # TODO: d0 should be a 3D vector, and accept negative values

a_vector = N.x * Piecewise((-a_magnitude, dynamicsymbols._t > T * R), (a_magnitude, True))
v0_vector = N.x * v0_magnitude
p0_vector = N.x * p0_magnitude

pT_magnitude = symbols('|d_{T}|', nonnegative=True) # TODO: pT should be a 3D vector, and accept negative values
pT_vector = N.x * pT_magnitude

vT_magnitude = symbols('|v_{T}|')
vT_vector = N.x * vT_magnitude

In [5]:
Markdown(rf"""
# Reference frame

$$N = \{{ {latex(N.x)}, {latex(N.y)}, {latex(N.z)} \}}$$

# Reference variables

$$
\begin{{align*}}
|a| & \quad & m.s^{{-2}} \quad & \text{{ Acceleration magnitude}} \\
v_0 &= {latex(v0_vector)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at }} t_0 \\
v_T &= {latex(vT_vector)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at destination}} \\
p_0 &= {latex(p0_vector)} \quad & m \quad & \text{{ Position at }} t_0 \\
p_T &= {latex(pT_vector)} \quad & m \quad & \text{{ Position at destination}}
\end{{align*}}
$$
""")


# Reference frame

$$N = \{ \mathbf{\hat{n}_x}, \mathbf{\hat{n}_y}, \mathbf{\hat{n}_z} \}$$

# Reference variables

$$
\begin{align*}
|a| & \quad & m.s^{-2} \quad & \text{ Acceleration magnitude} \\
v_0 &= |v_{0}|\mathbf{\hat{n}_x} \quad & m.s^{-1} \quad & \text{ Velocity at } t_0 \\
v_T &= |v_{T}|\mathbf{\hat{n}_x} \quad & m.s^{-1} \quad & \text{ Velocity at destination} \\
p_0 &= |p_{0}|\mathbf{\hat{n}_x} \quad & m \quad & \text{ Position at } t_0 \\
p_T &= |d_{T}|\mathbf{\hat{n}_x} \quad & m \quad & \text{ Position at destination}
\end{align*}
$$


In [26]:
def replace_min_T(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that min(t, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Min(t, R*T), R*T)

def replace_min_T_max_0_RT(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that min(t, max(0, R * T)) = R * T
    #  However sympy is not able to infer this, so we replace manually
#     return replace_min_T(f.replace(Max(sympify(0), R * T), R * T))
    return replace_min_T(f.replace(Max, lambda a, b: (b if a == 0 else a_)))

original_a_fn, original_v_fn, original_d_fn = get_motion_params(
    N,
    acceleration=a_vector, 
    velocity=v0_vector,
    position=p0_vector
)

a_fn = original_a_fn.subs(dynamicsymbols._t, t)
v_fn = original_v_fn.subs(dynamicsymbols._t, t).applyfunc(replace_min_T)
d_fn = original_d_fn.subs(dynamicsymbols._t, t).applyfunc(replace_min_T_max_0_RT)

In [7]:
Markdown(rf"""
# Motion parameters, with constant acceleration $a$

These formulae depend on:
* $T$ (in seconds): The total time to arrival
* $R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{{align}}
a(t) &= {latex(a_fn)} \\
v(t) &= {latex(v_fn)} \\
d(t) &= {latex(d_fn)}
\end{{align}}
$$
""")


# Motion parameters, with constant acceleration $a$

These formulae depend on:
* $T$ (in seconds): The total time to arrival
* $R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{align}
a(t) &= \begin{cases} - |a| & \text{for}\: t > R T \\|a| & \text{otherwise} \end{cases}\mathbf{\hat{n}_x} \\
v(t) &= (2 R T |a| - t |a| + |v_{0}|)\mathbf{\hat{n}_x} \\
d(t) &= (R^{2} T^{2} |a| + R T |v_{0}| - R T \left(2 R T |a| + |v_{0}|\right) - \frac{t^{2} |a|}{2} + t \left(2 R T |a| + |v_{0}|\right) + |p_{0}|)\mathbf{\hat{n}_x}
\end{align}
$$


In [8]:
eq_vT_x = Eq(
    vT_vector.to_matrix(N)[0],
    v_fn.subs(t, T).to_matrix(N)[0]
) # TODO: Make that equation on 3 dimensions

solved_R = list(solveset(eq_vT_x, R))[0] # The solveset returns a singleton, which is also a list of size 1

In [9]:
Markdown(rf"""
# Finding R

$R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{{align}}
v_T &= v(T) \\
{latex(vT_vector.to_matrix(N)[0])} &= {latex(v_fn.subs(t, T).to_matrix(N)[0])} \\
\end{{align}}
$$

This solves into:

$$R = {latex(solved_R)}$$
""")


# Finding R

$R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{align}
v_T &= v(T) \\
|v_{T}| &= 2 R T |a| - T |a| + |v_{0}| \\
\end{align}
$$

This solves into:

$$R = \frac{T |a| - |v_{0}| + |v_{T}|}{2 T |a|}$$


In [10]:
eq_dT_x = Eq(
    pT_vector.to_matrix(N)[0],
    d_fn.subs({t: T, R: solved_R}).to_matrix(N)[0]
) # TODO: Make that equation on 3 dimensions

# eq_dT_x solves as a quadratic expression, thus returns a finite set of two values.
solved_T1, solved_T2 = solveset(eq_dT_x, T)
# We need to turn these two values into a Piecewise, in order to substitute them
solved_T = Piecewise((solved_T1, solved_T1 >= 0), (solved_T2, True))

# TODO: Inject T >= 0 into the solveset to only get one result

In [17]:
Markdown(rf"""
# Finding T

$T$ (in seconds): The total time to arrival

$$
\begin{{align}}
p_T &= p(T) \\
{latex(pT_vector.to_matrix(N)[0])} &= {latex(d_fn.subs({t: T}).to_matrix(N)[0])} \\
\\
\text{{Given that }} R &= {latex(solved_R)} \\
{latex(pT_vector.to_matrix(N)[0])} &= {latex(d_fn.subs({t: T, R: solved_R}).to_matrix(N)[0])} \\
\end{{align}}
$$

This solves into:

$$T = {latex(solved_T)}$$
""")


# Finding T

$T$ (in seconds): The total time to arrival

$$
\begin{align}
p_T &= p(T) \\
|d_{T}| &= R^{2} T^{2} |a| + R T |v_{0}| - R T \left(2 R T |a| + |v_{0}|\right) - \frac{T^{2} |a|}{2} + T \left(2 R T |a| + |v_{0}|\right) + |p_{0}| \\
\\
\text{Given that } R &= \frac{T |a| - |v_{0}| + |v_{T}|}{2 T |a|} \\
|d_{T}| &= - \frac{T^{2} |a|}{2} + T \left(T |a| + |v_{T}|\right) + |p_{0}| + \frac{|v_{0}| \left(T |a| - |v_{0}| + |v_{T}|\right)}{2 |a|} - \frac{\left(T |a| + |v_{T}|\right) \left(T |a| - |v_{0}| + |v_{T}|\right)}{2 |a|} + \frac{\left(T |a| - |v_{0}| + |v_{T}|\right)^{2}}{4 |a|} \\
\end{align}
$$

This solves into:

$$T = \begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}$$


In [28]:
from sympy import piecewise_fold

a_fn_2 = a_fn.subs({T: solved_T, R: solved_R})
v_fn_2 = v_fn.subs({T: solved_T, R: solved_R})
d_fn_2 = d_fn.subs({T: solved_T, R: solved_R})

In [18]:
Markdown(rf"""
# Motion parameters, final equations

After injecting $T$ and $R$, we obtain:

$$
\begin{{align}}
a(t) &= {latex(a_fn_2)} \\
v(t) &= {latex(v_fn_2)} \\
d(t) &= {latex(d_fn_2)}
\end{{align}}
$$
""")


# Motion parameters, final equations

After injecting $T$ and $R$, we obtain:

$$
\begin{align}
a(t) &= \begin{cases} - |a| & \text{for}\: ITE\left(\frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0, t > \frac{|a| \left(- \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|}, t > \frac{|a| \left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|}\right) \\|a| & \text{otherwise} \end{cases}\mathbf{\hat{n}_x} \\
v(t) &= (- t |a| + |a| \left(\begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}\right) + |v_{T}|)\mathbf{\hat{n}_x} \\
d(t) &= (- \frac{t^{2} |a|}{2} + t \left(|a| \left(\begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}\right) + |v_{T}|\right) + |p_{0}| + \frac{|v_{0}| \left(|a| \left(\begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}\right) - |v_{0}| + |v_{T}|\right)}{2 |a|} - \frac{\left(|a| \left(\begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}\right) + |v_{T}|\right) \left(|a| \left(\begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}\right) - |v_{0}| + |v_{T}|\right)}{2 |a|} + \frac{\left(|a| \left(\begin{cases} - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{for}\: \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} \leq 0 \\- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|} & \text{otherwise} \end{cases}\right) - |v_{0}| + |v_{T}|\right)^{2}}{4 |a|})\mathbf{\hat{n}_x}
\end{align}
$$


In [14]:
def get_t_vals(args):
    max_T = float(solved_T.subs(args))
    return np.linspace(0, max_T, 99)

def plot_position(a_knob, v0_knob, p0_knob, pT_knob, vT_knob):
    args = {a_magnitude: a_knob, v0_magnitude: v0_knob, p0_magnitude: p0_knob, pT_magnitude: pT_knob, vT_magnitude: vT_knob}
    t_vals = get_t_vals(args)
    
    d = d_fn_2.subs(args).to_matrix(N)[0]
    position = go.Figure()
    position.add_trace(go.Scatter(x=list(t_vals), y=[float(d.subs(t, t_val)) for t_val in t_vals], line_shape='spline', name="Position(x)"))

    position.update_layout(title="Position(x)", xaxis_title="t", yaxis_title="p(t)")
    position.show()
    
def plot_velocity(a_knob, v0_knob, p0_knob, pT_knob, vT_knob):
    args = {a_magnitude: a_knob, v0_magnitude: v0_knob, p0_magnitude: p0_knob, pT_magnitude: pT_knob, vT_magnitude: vT_knob}
    t_vals = get_t_vals(args)
    
    v = v_fn_2.subs(args).to_matrix(N)[0]
    position = go.Figure()
    position.add_trace(go.Scatter(x=list(t_vals), y=[float(v.subs(t, t_val)) for t_val in t_vals], line_shape='spline', name="Velocity(x)"))

    position.update_layout(title="Velocity(t)", xaxis_title="t", yaxis_title="v(t)")
    position.show()
    
def plot_acceleration(a_knob, v0_knob, p0_knob, pT_knob, vT_knob):
    args = {a_magnitude: a_knob, v0_magnitude: v0_knob, p0_magnitude: p0_knob, pT_magnitude: pT_knob, vT_magnitude: vT_knob}
    t_vals = get_t_vals(args)

    a = a_fn_2.subs(args).to_matrix(N)[0]
    position = go.Figure()
    position.add_trace(go.Scatter(x=list(t_vals), y=[float(a.subs(t, t_val)) for t_val in t_vals], line_shape='hv', name="Acceleration(x)"))

    position.update_layout(title="Acceleration(t)", xaxis_title="t", yaxis_title="a(t)")
    position.show()

In [15]:
import ipywidgets as widgets
from ipywidgets import interactive_output

a_knob = widgets.BoundedFloatText(value=1, min=0.1, max=15.0, step=0.1, description='Acceleration (m.s-2)')
v0_knob = widgets.IntText(value=0, step=10, description='v0 (m.s-1)')
vT_knob = widgets.IntText(value=0, step=10, description='vT (m.s-1)')
p0_knob = widgets.IntText(value=0, step=10, description='p0 (m)')
pT_knob = widgets.IntText(value=100000, step=10, description='pT (m)')

ui = widgets.GridBox([a_knob, v0_knob, vT_knob, p0_knob, pT_knob])
display(ui)

GridBox(children=(BoundedFloatText(value=1.0, description='Acceleration (m.s-2)', max=15.0, min=0.1, step=0.1)…

In [16]:
knob_args = {"a_knob": a_knob, "v0_knob": v0_knob, "p0_knob": p0_knob, "pT_knob": pT_knob, "vT_knob": vT_knob}

position_plot = interactive_output(plot_position, knob_args)
velocity_plot = interactive_output(plot_velocity, knob_args)
acceleration_plot = interactive_output(plot_acceleration, knob_args)
display(position_plot, velocity_plot, acceleration_plot)

Output()

Output()

Output()